In [1]:
import pandas as pd
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

## 一、测试LangChain内置代码解释器工具功能

In [10]:
dat = pd.read_csv("data/WA_Fn-UseC_-Telco-Customer-Churn.csv",header=0)

from langchain_experimental.tools import PythonAstREPLTool
# langchain内置的python解释器，将其功能传递给python_tool变量。
python_tool = PythonAstREPLTool(locals={"df":dat})
python_tool.invoke("df['SeniorCitizen'].mean()")

model = ChatOllama(
    model = 'llama3.1:8b',
    # model = 'deepseek-r1:1.5b', 没有调用tool功能
    base_url = 'http://localhost:11434/'
)

In [11]:
model

ChatOllama(model='llama3.1:8b', base_url='http://localhost:11434/')

In [12]:
# bind_tools 是一个用于将工具（Tools）绑定到可调用对象（如 LLM 或 Chain）上的方法，使得模型能够更好地与外部工具交互。
model_with_tool = model.bind_tools([python_tool])
response = model_with_tool.invoke("请分析'df'这张表，计算有数值的列的平均值。")

In [13]:
# 查看response输出，可以看到content内容是空的
# ✅ 模型 确实识别并调用了绑定的工具（python_repl_ast）
# ✅ df.describe() 是模型产生的代码
# ❌ 模型没有返回“人类可读的回答”，即 content 是空的
# 调用的是 model_with_tool.invoke(...)，只触发 tool_call，但未执行该工具。
response

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-11T07:02:28.995452Z', 'done': True, 'done_reason': 'stop', 'total_duration': 29946262400, 'load_duration': 7621630600, 'prompt_eval_count': 216, 'prompt_eval_duration': 11186185000, 'eval_count': 46, 'eval_duration': 11131381400, 'model_name': 'llama3.1:8b'}, id='run--b7963682-daf5-4bb9-8dec-da1338bfba27-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'query': "import pandas as pd\npd.DataFrame({'A': [1, 2], 'B': [3, 4]}).describe()"}, 'id': 'eb2e7072-c1fd-4e5f-afa0-43acfd4248cd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 216, 'output_tokens': 46, 'total_tokens': 262})

In [16]:
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser
parser = JsonOutputKeyToolsParser(key_name=python_tool.name, first_tool_only=True)
model_chain = model_with_tool | parser
response = model_chain.invoke("请分析'df'这张表，计算有数值的列的平均值。")
response

{'query': "import pandas as pd\npd.set_option('display.max_rows', None)\ndf.describe()"}

In [18]:
# JsonOutputKeyToolsParser 从大模型的 function calling / tool calling 结构中，提取你关心的字段（key），并转换为 Python dict，供下一步使用（比如代码执行）。
print(parser)
print(model_chain)

first_tool_only=True key_name='python_repl_ast'
first=RunnableBinding(bound=ChatOllama(model='llama3.1:8b', base_url='http://localhost:11434/'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'python_repl_ast', 'description': 'A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.', 'parameters': {'properties': {'query': {'description': 'code snippet to run', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]}, config={}, config_factories=[]) middle=[] last=JsonOutputKeyToolsParser(first_tool_only=True, key_name='python_repl_ast')


In [19]:
# 所以接下来给出提示词模板，告知大模型需要操作
system = f"""
你可以访问一个名为 `df` 的 pandas 数据框，你可以使用df.head().to_markdown() 查看数据集的基本信息， \
请根据用户提出的问题，编写 Python 代码来回答。只返回代码，不返回其他内容。只允许使用 pandas 和内置库。
"""
prompt = ChatPromptTemplate([
    ("system", system),
    ("user", "{question}")
])
code_chain = prompt | model_with_tool | parser

通过 prompt <font color=red> 向模型提问 → 模型识别你希望它生成可执行代码 → 模型以 Tool 调用的形式返回 {"query": "..."} → JsonOutputKeyToolsParser 负责提取出这段代码</font></br>
最终输出了：{'query': "df.select_dtypes(include=['number']).mean()"}

In [20]:
code_chain.invoke({"question": "请分析'df'这张表，计算有数值的列的平均值。"})

{'query': 'df.describe()'}

In [21]:
chain = code_chain | python_tool
chain.invoke({"question": "请分析'df'这张表，计算有数值的列的平均值。"})

'SeniorCitizen      0.162147\ntenure            32.371149\nMonthlyCharges    64.761692\ndtype: float64\n'

In [23]:
chain.invoke({"question": "分析'df'这张表，gender统计信息，每个类别的总数"})

'gender\nMale      3555\nFemale    3488\nName: count, dtype: int64\n'